In [53]:
import numpy as np
import jsonlines
import json
import pandas as pd
import os
from tqdm import tqdm
import sys
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/workspace/swe-data/models/GLM-4.7-template/", trust_remote_code=True)

In [29]:
def normalize_tool_calls(messages):
    for m in messages:
        if "tool_calls" in m:
            for tc in m["tool_calls"]:
                func = tc.get("function") or tc
                args = func.get("arguments")
                if isinstance(args, str):
                    try:
                        func["arguments"] = json.loads(args)
                    except Exception as e:
                        print(f'Error loading arguments: {e}', flush=True)
                        pass
    return messages

In [42]:
base_path = "/workspace/haoran-cloud/code/kimi-cli/results/GLM-4.7_20260111_143759/"
output_file = f'{base_path}/results.jsonl'
with jsonlines.open(output_file) as reader:
    results = list(reader)

In [50]:
stats = []
for data in tqdm(results):
    try:
        messages = [{"id": "main", "messages": normalize_tool_calls(data['messages'])}]
        sub_messages = data['sub_messages']
        for sub_item in sub_messages:
            sub_item["messages"] = normalize_tool_calls(sub_item["messages"])
        total_messsages = messages + sub_messages
        data["has_sub_agent"] = len(sub_messages) > 0
        data["main_traj_tokens"] = len(tokenizer.apply_chat_template(messages[0]["messages"]))
        data["main_traj_turns"] = len(messages[0]["messages"])
        data["sub_traj_tokens"] = []
        data["sub_traj_turns"] = []
        for sub_message in sub_messages:
            sub_traj_tokens = len(tokenizer.apply_chat_template(sub_message["messages"]))
            sub_traj_turns = len(sub_message["messages"])
            data["sub_traj_tokens"].append(sub_traj_tokens)
            data["sub_traj_turns"].append(sub_traj_turns)
        stats.append(data)
    except Exception as e:
        print(e)
        continue

 53%|█████▎    | 54/102 [00:04<00:03, 12.46it/s]

list index out of range


 85%|████████▌ | 87/102 [00:07<00:01,  9.21it/s]

Error loading arguments: Unterminated string starting at: line 1 column 56 (char 55)
Error loading arguments: Unterminated string starting at: line 1 column 56 (char 55)
Error loading arguments: Unterminated string starting at: line 1 column 57 (char 56)
Error loading arguments: Unterminated string starting at: line 1 column 56 (char 55)


 89%|████████▉ | 91/102 [00:08<00:00, 13.26it/s]

...

Error loading arguments: Unterminated string starting at: line 1 column 64 (char 63)
Error loading arguments: Unterminated string starting at: line 1 column 62 (char 61)
Error loading arguments: Unterminated string starting at: line 1 column 60 (char 59)
Error loading arguments: Unterminated string starting at: line 1 column 12 (char 11)
Error loading arguments: Unterminated string starting at: line 1 column 60 (char 59)
Error loading arguments: Unterminated string starting at: line 1 column 61 (char 60)
Error loading arguments: Unterminated string starting at: line 1 column 64 (char 63)
Error loading arguments: Unterminated string starting at: line 1 column 63 (char 62)
Error loading arguments: Unterminated string starting at: line 1 column 63 (char 62)
Error loading arguments: Unterminated string starting at: line 1 column 12 (char 11)
Error loading arguments: Unterminated string starting at: line 1 column 64 (char 63)


 97%|█████████▋| 99/102 [00:08<00:00, 11.27it/s]

'str object' has no attribute 'items'
Error loading arguments: Unterminated string starting at: line 1 column 88 (char 87)
'str object' has no attribute 'items'
Error loading arguments: Unterminated string starting at: line 1 column 67 (char 66)
Error loading arguments: Unterminated string starting at: line 1 column 67 (char 66)
Error loading arguments: Unterminated string starting at: line 1 column 67 (char 66)
Error loading arguments: Unterminated string starting at: line 1 column 67 (char 66)
Error loading arguments: Unterminated string starting at: line 1 column 83 (char 82)
Error loading arguments: Unterminated string starting at: line 1 column 83 (char 82)


100%|██████████| 102/102 [00:08<00:00, 11.40it/s]

'str object' has no attribute 'items'


In [54]:
stats = pd.DataFrame(stats)

In [58]:
print(stats["main_traj_tokens"].max(), stats["main_traj_tokens"].min(), stats["main_traj_tokens"].mean())
print(stats["main_traj_turns"].max(), stats["main_traj_turns"].min(), stats["main_traj_turns"].mean())
# sum for sub_traj_tokens and sub_traj_turns
print(stats["sub_traj_tokens"].sum(), stats["sub_traj_turns"].sum())
print(stats["has_sub_agent"].value_counts())


74187 3317 44279.54639175258
333 6 130.76288659793815
[46068, 20778, 53476, 59430, 55330, 5336, 71142, 50810, 72013, 7661, 18934, 71167, 47594, 55629, 14967, 6169, 73345, 8442, 19769, 4648, 70948] [37, 35, 112, 84, 104, 9, 111, 229, 119, 16, 43, 110, 86, 232, 33, 13, 122, 26, 99, 13, 123]
has_sub_agent
False    85
True     12
Name: count, dtype: int64
